In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# importamos librerías necesarias 
import pandas as pd
import json
import requests
import os # no hay que instalarlo
from dotenv import load_dotenv # esta es una librería para poder encriptar nuestra api_key

In [4]:
# 1) Abrimos la “caja” (el archivo JSON)
with open("definitivo.json", "r") as file:
    canciones = json.load(file)

In [5]:
flamenco_2010_2015 = []

for c in canciones:
    genero = str(c.get("genre", "")).lower()

    try:
        year = int(c.get("year"))
    except:
        continue  # si no se puede convertir a número, saltamos esa canción

    if genero == "flamenco" and 2010 <= year <= 2015:
        flamenco_2010_2015.append(c)

In [6]:
flamenco_2010_2015

[{'id': '1UU0VkPwhyiYyO7PT0dyqU',
  'artist_name': 'Angelillo',
  'track_name': 'El Ruiseñor Herido',
  'genre': 'flamenco',
  'year': 2010},
 {'id': '1UU0VkPwhyiYyO7PT0dyqU',
  'artist_name': 'Maestro Quiroga',
  'track_name': 'El Ruiseñor Herido',
  'genre': 'flamenco',
  'year': 2010},
 {'id': '1UU0VkPwhyiYyO7PT0dyqU',
  'artist_name': 'Orchestra Acroma',
  'track_name': 'El Ruiseñor Herido',
  'genre': 'flamenco',
  'year': 2010},
 {'id': '0apMtgST1WVuiiYIdQdO1P',
  'artist_name': 'Juanito Valderrama',
  'track_name': 'El Emigrante',
  'genre': 'flamenco',
  'year': 2010},
 {'id': '08sMOLGjPFgFtCrf0EvN3V',
  'artist_name': 'Antoñita Moreno',
  'track_name': 'Carretera De Asturias',
  'genre': 'flamenco',
  'year': 2010},
 {'id': '4nafSzGGjl0X4sfGIqRvyh',
  'artist_name': 'La Niña De La Puebla',
  'track_name': 'Pasodoble Con Fandangos',
  'genre': 'flamenco',
  'year': 2010},
 {'id': '5e1CIVCBsSv9NX5yfGGvWj',
  'artist_name': 'Juanito Valderrama',
  'track_name': 'Su Primera Comuni

In [7]:
artistas_set = set()

for c in flamenco_2010_2015:
    nombre = c.get("artist_name")
    if nombre:
        artistas_set.add(nombre.strip())

artistas = sorted(artistas_set)
print("Artistas únicos:", len(artistas))

Artistas únicos: 113


BUSCAMOS ARTISTA EN LAST.FM

In [8]:
# llamada de prueba a Last.fm (con un artista)


load_dotenv(".env")

api_key = os.getenv("lastFM_api_key")


In [9]:
#FUNCIÓN PARA PEDIR A LAST.FM
BASE_URL = "https://ws.audioscrobbler.com/2.0/"


def pedir_a_lastfm(params):
    params["api_key"] = api_key
    params["format"] = "json"
    r = requests.get(BASE_URL, params=params)
    return r.json()



In [10]:
# prueba de la función con un artista
respuesta_info = pedir_a_lastfm(
    {
        "method": "artist.getinfo",
        "artist": "Juanito Valderrama",
        "autocorrect": 1,
        "lang": "es",
    }
)
print(respuesta_info.keys())


# Sacar campos de la respuesta INFO
artista = respuesta_info["artist"]

nombre = artista.get("name")
listeners = (artista.get("stats") or {}).get("listeners")
playcount = (artista.get("stats") or {}).get("playcount")
bio = (artista.get("bio") or {}).get("summary")  # resumen

dict_keys(['artist'])


In [11]:
# sacar campo de artistas similares
respuesta_similares = pedir_a_lastfm({
    "method": "artist.getsimilar",
    "artist": "Juanito Valderrama",
    "autocorrect": 1,
    "limit": 5, })  # cuantos similares quieres

In [12]:
lista_similares = [
    a.get("name")
    for a in (respuesta_similares.get("similarartists") or {}).get("artist", [])
    if isinstance(a, dict) and a.get("name")
]

In [13]:
#  Creo la “ficha” final (limpia) con los campos que me interesan
ficha = {
    "artist_name": nombre,
    "bio": bio,
    "listeners": listeners,
    "playcount": playcount,
    "similar_artists": lista_similares,
}

print(ficha)

{'artist_name': 'Juanito Valderrama', 'bio': 'Juan Valderrama Blanca, más conocido como Juanito Valderrama, (24 de mayo, 1916, Torredelcampo, Jaén – 12 de abril, 2004, Espartinas, Sevilla) fue un cantaor de flamenco español.\n\nNacido en el seno de una familia de agricultores, la carrera profesional de Valderrama comenzó en 1935, en el Cine Metropolitano de Madrid, y se prolongaría más de 60 años. Sin embargo, se había iniciado en el flamenco desde niño, mientras trabajaba en el campo. <a href="https://www.last.fm/music/Juanito+Valderrama">Read more on Last.fm</a>', 'listeners': '19593', 'playcount': '81774', 'similar_artists': ['Rafael Farina', 'Antonio Molina', 'Juanita Reina', 'Concha Piquer', 'Marifé De Triana']}


In [ ]:
bio_limpia = ficha["bio"]

if bio_limpia:
    # cortar antes del link (si existe)
    bio_limpia = bio_limpia.split("<a href=")[0]

    # quitar el texto “Read more...” si quedara
    bio_limpia = bio_limpia.replace("Read more on Last.fm", "")

    # cambiar saltos de línea por espacios
    bio_limpia = bio_limpia.replace("\n", " ")

    # quitar espacios dobles que puedan quedar
    bio_limpia = " ".join(bio_limpia.split())

ficha["bio"] = bio_limpia
print(ficha["bio"])

Juan Valderrama Blanca, más conocido como Juanito Valderrama, (24 de mayo, 1916, Torredelcampo, Jaén – 12 de abril, 2004, Espartinas, Sevilla) fue un cantaor de flamenco español. Nacido en el seno de una familia de agricultores, la carrera profesional de Valderrama comenzó en 1935, en el Cine Metropolitano de Madrid, y se prolongaría más de 60 años. Sin embargo, se había iniciado en el flamenco desde niño, mientras trabajaba en el campo.


In [17]:
print(len(artistas)) 


113


In [ ]:
import time


# --- Limpieza de bio 
def limpiar_bio_op1(bio):
    if not bio:
        return None
    bio = bio.split("<a href=")[0]
    bio = bio.replace("Read more on Last.fm", "")
    bio = bio.replace("\n", " ")
    bio = " ".join(bio.split())
    return bio


fichas = []

for i, artista_nombre in enumerate(artistas, start=1):
    try:
        # 1) INFO
        info = pedir_a_lastfm(
            {
                "method": "artist.getinfo",
                "artist": artista_nombre,
                "autocorrect": 1,
                "lang": "es",
            }
        )

        # 2) SIMILARES
        sim = pedir_a_lastfm(
            {
                "method": "artist.getsimilar",
                "artist": artista_nombre,
                "autocorrect": 1,
                "limit": 10,
            }
        )

        # 3) Sacar campos
        a = info.get("artist", {})

        nombre = a.get("name") or artista_nombre
        listeners = (a.get("stats") or {}).get("listeners")
        playcount = (a.get("stats") or {}).get("playcount")

        bio = (a.get("bio") or {}).get("summary")
        bio = limpiar_bio_op1(bio)

        lista_similares = [
            x.get("name")
            for x in (sim.get("similarartists") or {}).get("artist", [])
            if isinstance(x, dict) and x.get("name")
        ]

        ficha = {
            "artist_name": nombre,
            "bio": bio,
            "listeners": listeners,
            "playcount": playcount,
            "similar_artists": lista_similares,
        }

        fichas.append(ficha)
        print(f"✅ {i}/{len(artistas)} -> {nombre}")

    except Exception as e:
        fichas.append({"artist_name": artista_nombre, "error": str(e)})
        print(f"❌ {i}/{len(artistas)} -> {artista_nombre} ERROR: {e}")

    time.sleep(0.25)

# --- QUITAR DUPLICADOS por artist_name (se queda el primero) ---
unicas = {}
for f in fichas:
    name = (f.get("artist_name") or "").strip().lower()
    if not name:
        continue
    if name not in unicas:
        unicas[name] = f

fichas = list(unicas.values())

print("Total fichas (sin duplicados):", len(fichas))
print("Ejemplo:", fichas[:1])

✅ 1/112 -> Agujetas Chico
✅ 2/112 -> Alazan
✅ 3/112 -> Almasäla
✅ 4/112 -> Andres Fernandez Amador
✅ 5/112 -> Andrés Hernández Pituquete
✅ 6/112 -> Angelillo
✅ 7/112 -> Antonio Rey
✅ 8/112 -> Antonio Reyes
✅ 9/112 -> Antoñita Moreno
✅ 10/112 -> Antoñita Peñuela
✅ 11/112 -> Asustadizo
✅ 12/112 -> Bordon-4
✅ 13/112 -> Camarón de la Isla
✅ 14/112 -> Camela
✅ 15/112 -> Canelita
✅ 16/112 -> Canteca De Macao
✅ 17/112 -> Carlos Montoya
✅ 18/112 -> Carlos Verdeal
✅ 19/112 -> Chano Domínguez
✅ 20/112 -> Cherokee
✅ 21/112 -> Damaris Martinez
✅ 22/112 -> David DeMaría
✅ 23/112 -> Diego Castellon
✅ 24/112 -> Diego Del Morao
✅ 25/112 -> Duo Joncol
✅ 26/112 -> Ecos del Rocío
✅ 27/112 -> El Loreño
✅ 28/112 -> Electro Rumbaiao
✅ 29/112 -> Encarna Anillo
✅ 30/112 -> Enrique Heredia Negri
✅ 31/112 -> Enrique Morente
✅ 32/112 -> Esteban De Sanlucar
✅ 33/112 -> Estrella Morente
✅ 34/112 -> Ezequiel Benítez
✅ 35/112 -> Flamencopatia
✅ 36/112 -> Gracia Montes
✅ 37/112 -> Grupo Macarena
✅ 38/112 -> Hermano R

## VAMOS A DEJAR LAS TABLAS BONITAS CON PANDAS

In [21]:
import pandas as pd

TABLA DE ARTISTAS (LAST.FM)

In [35]:
# convertimos la lista de fichas a un DataFrame de pandas
df_artistas = pd.DataFrame(fichas)

In [36]:
df_artistas

,artist_name,bio,listeners,playcount,similar_artists
0,Agujetas Chico,,61,429,"[Manuel Parrilla, Los Moneo, Momo de Jerez, je..."
1,Alazan,,3232,15365,"[La Húngara, Selena Leo, Raya Real, Dani Mata,..."
2,Almasäla,"Cantante, compositora, bailaora, polifacética ...",328,2117,"[Eskorzo Y Albert Pla, Costo Rico, Quinto Parp..."
3,Andres Fernandez Amador,,10391,37613,"[Rafa El Tachuela, Dani de Morón, Juan Manuel ..."
4,Andrés Hernández Pituquete,,57,521,"[Moraíto, Rafael Riqueni, Manolo Sanlucar, Jua..."
...,...,...,...,...,...
107,Soniquete,Con el fin de promover el flamenco entre los m...,996,6090,"[Raya Real, Kiki Morente, juanlu montoya, Isra..."
108,Sonsonete,,1326,9633,"[LOS BANIS, Nouna, aldeskuido, Currichi, Parri..."
109,Tomatito,Tomatito es el nombre artístico de José Fernán...,78263,504592,"[Vicente Amigo, Paco de Lucía, Estrella Morent..."
110,Trikitown,,357,1256,"[Vifidus, Patriarcas De La Rumba, Los Impagaos..."


In [37]:
# Si existe columna error, nos quedamos solo con las fichas buenas
if "error" in df_artistas.columns:
    df_artistas = df_artistas[df_artistas["error"].isna()].copy()

In [27]:
df["similar_artists"] = df["similar_artists"].apply(
    lambda x: "; ".join(x) if isinstance(x, list) else ""
)

In [38]:
# Si la bio falta (None/NaN), que salga "" (vacío)
df_artistas["bio"] = df_artistas["bio"].fillna("")

In [ ]:
# Similar artists: de lista -> texto bonito (para que se vea bien en tabla/CSV)
df_artistas["similar_artists"] = df_artistas["similar_artists"].apply(
    lambda x: "; ".join(x) if isinstance(x, list) else ""
)

In [41]:
print(type(df_artistas["listeners"].iloc[0]))
type(df_artistas["playcount"].iloc[0])

<class 'str'>


str

In [42]:
# Convertir a número para ordenar bien
df_artistas["listeners"] = pd.to_numeric(df_artistas["listeners"], errors="coerce")
df_artistas["playcount"] = pd.to_numeric(df_artistas["playcount"], errors="coerce")

In [43]:
df_artistas

,artist_name,bio,listeners,playcount,similar_artists
0,Agujetas Chico,,61.0,429.0,"[Manuel Parrilla, Los Moneo, Momo de Jerez, je..."
1,Alazan,,3232.0,15365.0,"[La Húngara, Selena Leo, Raya Real, Dani Mata,..."
2,Almasäla,"Cantante, compositora, bailaora, polifacética ...",328.0,2117.0,"[Eskorzo Y Albert Pla, Costo Rico, Quinto Parp..."
3,Andres Fernandez Amador,,10391.0,37613.0,"[Rafa El Tachuela, Dani de Morón, Juan Manuel ..."
4,Andrés Hernández Pituquete,,57.0,521.0,"[Moraíto, Rafael Riqueni, Manolo Sanlucar, Jua..."
...,...,...,...,...,...
107,Soniquete,Con el fin de promover el flamenco entre los m...,996.0,6090.0,"[Raya Real, Kiki Morente, juanlu montoya, Isra..."
108,Sonsonete,,1326.0,9633.0,"[LOS BANIS, Nouna, aldeskuido, Currichi, Parri..."
109,Tomatito,Tomatito es el nombre artístico de José Fernán...,78263.0,504592.0,"[Vicente Amigo, Paco de Lucía, Estrella Morent..."
110,Trikitown,,357.0,1256.0,"[Vifidus, Patriarcas De La Rumba, Los Impagaos..."


TABLA SPOTIFY GENERO FLAMENCO 2010-2015

In [57]:
df_canciones = pd.DataFrame(flamenco_2010_2015)

In [58]:
df_canciones

,id,artist_name,track_name,genre,year
0,1UU0VkPwhyiYyO7PT0dyqU,Angelillo,El Ruiseñor Herido,flamenco,2010
1,1UU0VkPwhyiYyO7PT0dyqU,Maestro Quiroga,El Ruiseñor Herido,flamenco,2010
2,1UU0VkPwhyiYyO7PT0dyqU,Orchestra Acroma,El Ruiseñor Herido,flamenco,2010
3,0apMtgST1WVuiiYIdQdO1P,Juanito Valderrama,El Emigrante,flamenco,2010
4,08sMOLGjPFgFtCrf0EvN3V,Antoñita Moreno,Carretera De Asturias,flamenco,2010
...,...,...,...,...,...
520,7vug6hR9K8DohUN6cBNMdj,Agujetas Chico,Soleá,flamenco,2015
521,7Es6aG5j8ykG7wPZ1GAwsj,Camaron De La Isla,Y No Llegaste A Quererme - Granaína,flamenco,2015
522,7Es6aG5j8ykG7wPZ1GAwsj,Paco de Lucía,Y No Llegaste A Quererme - Granaína,flamenco,2015
523,738QDJ3pidmz7elZ04tXyl,Agujetas Chico,Bulería pa Escuchá,flamenco,2015


In [59]:
# Renombrar columnas para que coincidan con el enunciado
df_canciones = df_canciones.rename(
    columns={
        "artist_name": "Artista",
        "genre": "Género musical",
        "track_name": "Nombre de la canción",
        "year": "Año de lanzamiento",
        "id": "ID",
    }
)

In [60]:
df_canciones

,ID,Artista,Nombre de la canción,Género musical,Año de lanzamiento
0,1UU0VkPwhyiYyO7PT0dyqU,Angelillo,El Ruiseñor Herido,flamenco,2010
1,1UU0VkPwhyiYyO7PT0dyqU,Maestro Quiroga,El Ruiseñor Herido,flamenco,2010
2,1UU0VkPwhyiYyO7PT0dyqU,Orchestra Acroma,El Ruiseñor Herido,flamenco,2010
3,0apMtgST1WVuiiYIdQdO1P,Juanito Valderrama,El Emigrante,flamenco,2010
4,08sMOLGjPFgFtCrf0EvN3V,Antoñita Moreno,Carretera De Asturias,flamenco,2010
...,...,...,...,...,...
520,7vug6hR9K8DohUN6cBNMdj,Agujetas Chico,Soleá,flamenco,2015
521,7Es6aG5j8ykG7wPZ1GAwsj,Camaron De La Isla,Y No Llegaste A Quererme - Granaína,flamenco,2015
522,7Es6aG5j8ykG7wPZ1GAwsj,Paco de Lucía,Y No Llegaste A Quererme - Granaína,flamenco,2015
523,738QDJ3pidmz7elZ04tXyl,Agujetas Chico,Bulería pa Escuchá,flamenco,2015
